In [26]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import dotenv
import os
import requests

In [22]:
BASE_URL = "https://hacker-news.firebaseio.com/v0/"

def get_data(endpoint): 
    URL = f"{BASE_URL}{endpoint}.json"
    response = requests.get(URL, timeout=10)
    return response.json()

@tool
def get_top_story_ids(limit=10):
    """
    Get the top {limit} story ids from Hacker News
    """
    return get_data("topstories")[:limit]

@tool
def get_story(story_id):
    """
    Get the data of any given post on Hacker News
    Data includes title, url, score, by, and time
    """
    return get_data(f"item/{story_id}")

In [23]:
top_ids = get_top_story_ids.invoke({"limit": 10})
for idx, story_id in enumerate(top_ids, start=1):
    item = get_story.invoke({"story_id": top_ids[0]})
    title = item.get("title", "<no title>")
    url = item.get("url", f"https://news.ycombinator.com/item?id={story_id}")
    score = item.get("score", 0)
    by = item.get("by", "unknown")
    print(f"{idx}. {title} ({score} points by {by})")
    print(f"{url}")

1. Meta's Legal Team Abandoned Its Ethical Duties (196 points by shrubby)
https://www.afterbabel.com/p/how-metas-lawyers-perfected-the-playbook
2. Meta's Legal Team Abandoned Its Ethical Duties (196 points by shrubby)
https://www.afterbabel.com/p/how-metas-lawyers-perfected-the-playbook
3. Meta's Legal Team Abandoned Its Ethical Duties (196 points by shrubby)
https://www.afterbabel.com/p/how-metas-lawyers-perfected-the-playbook
4. Meta's Legal Team Abandoned Its Ethical Duties (196 points by shrubby)
https://www.afterbabel.com/p/how-metas-lawyers-perfected-the-playbook
5. Meta's Legal Team Abandoned Its Ethical Duties (196 points by shrubby)
https://www.afterbabel.com/p/how-metas-lawyers-perfected-the-playbook
6. Meta's Legal Team Abandoned Its Ethical Duties (196 points by shrubby)
https://www.afterbabel.com/p/how-metas-lawyers-perfected-the-playbook
7. Meta's Legal Team Abandoned Its Ethical Duties (196 points by shrubby)
https://www.afterbabel.com/p/how-metas-lawyers-perfected-the-p

## Let's create the agent

In [ ]:
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(api_key=OPENAI_API_KEY)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about Hacker News."),
    ("human", "{input}"), 
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

tools = [get_top_story_ids, get_story]

agent = create_openai_functions_agent(llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

result = agent_executor.invoke({"input": "What are the top trends in the recent Hacker News posts?"})
print(result["output"])
print(result["intermediate_steps"])



> Entering new AgentExecutor chain...

Invoking: `get_top_story_ids` with `{'limit': 5}`


[46694378, 46694124, 46691835, 46693346, 46693828]
Invoking: `get_story` with `{'story_id': 46694378}`


{'by': 'shrubby', 'descendants': 108, 'id': 46694378, 'kids': [46694847, 46695096, 46695260, 46695098, 46695505, 46695343, 46695179, 46695373, 46695287, 46695041, 46694899, 46694835, 46695405, 46695448, 46695561, 46695032, 46695049, 46694886, 46694917, 46694739, 46694861], 'score': 197, 'time': 1768928545, 'title': "Meta's Legal Team Abandoned Its Ethical Duties", 'type': 'story', 'url': 'https://www.afterbabel.com/p/how-metas-lawyers-perfected-the-playbook'}
Invoking: `get_story` with `{'story_id': 46694124}`


{'by': 'kykeonaut', 'descendants': 11, 'id': 46694124, 'kids': [46694993, 46695420, 46695602, 46694887, 46694611, 46694501, 46694372, 46695089], 'score': 36, 'time': 1768927739, 'title': 'The Unix Pipe Card Game', 'type': 'story', 'url': 'https://punkx.org/unix-pipe-game/'}